In [ ]:
try:
    import sim
except:
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time
import numpy as np


def normalizeAngle(angle):
    return np.mod(angle+np.pi, 2*np.pi) - np.pi


def collinear(x1, y1, x2, y2, x3, y3):
    a = x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)
    print(a)
    if (a <= 0.5 and a >= 0):
        return True
    elif (a >= -0.5 and a <= 0):
        return True
    return False

In [ ]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID != -1:
    print ('Connected to remote API server')

    robotname = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)     
        
    returnCode, l_wheel = sim.simxGetObjectHandle(clientID, robotname + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, r_wheel = sim.simxGetObjectHandle(clientID, robotname + '_rightMotor', sim.simx_opmode_oneshot_wait)    
    
    # Handles para os sonares
    returnCode, sonar_front = sim.simxGetObjectHandle(clientID, robotname + '_ultrasonicSensor5', sim.simx_opmode_oneshot_wait)
    returnCode, sonar_right = sim.simxGetObjectHandle(clientID, robotname + '_ultrasonicSensor7', sim.simx_opmode_oneshot_wait)

    # Goal config
    qgoal = np.array([-2, 3, np.deg2rad(90)])

    # Frame que representa o Goal
    returnCode, goalFrame = sim.simxGetObjectHandle(clientID, 'Goal', sim.simx_opmode_oneshot_wait)     
    returnCode = sim.simxSetObjectPosition(clientID, goalFrame, -1, [qgoal[0], qgoal[1], 0], sim.simx_opmode_oneshot_wait)
    returnCode = sim.simxSetObjectOrientation(clientID, goalFrame, -1, [0, 0, qgoal[2]], sim.simx_opmode_oneshot_wait)    
       
    # Específico do robô   
    L = 0.331
    r = 0.09751
    maxv = 1.0
    maxw = np.deg2rad(45)
    
    following = False
    err = np.inf
    it = 0
    while err > .05:
        # Pos, Ori
        returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)  
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])

        dx, dy, dth = qgoal - robotConfig
        err = np.sqrt(dx**2 + dy**2)
        alpha = normalizeAngle(-robotConfig[2] + np.arctan2(dy,dx))
        beta = normalizeAngle(qgoal[2] - np.arctan2(dy,dx))

        # Fazendo leitura dos sensores
        returnCode, detected_front, point_front, *_ = sim.simxReadProximitySensor(clientID, sonar_front, sim.simx_opmode_oneshot_wait)
        returnCode, detected_right, point_right, *_ = sim.simxReadProximitySensor(clientID, sonar_right, sim.simx_opmode_oneshot_wait)
        
        kr = 4 / 20
        ka = 8 / 20
        kb = -1.5 / 20

        if abs(alpha) > np.pi/2:
            kr = -kr       
            # Se não ajustar a direção muda
            alpha = normalizeAngle(alpha - np.pi)
            beta = normalizeAngle(beta - np.pi)
        
        v = kr * err
        w = ka * alpha + kb * beta

        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw) 

        obstacle_in_front = (detected_front and np.linalg.norm(point_front) < .9)
        obstacle_in_right = (detected_right and np.linalg.norm(point_right) < .9)

        # Controle
        if obstacle_in_front:
            v = 0
            w = np.deg2rad(30)
            following = True
        else: 
            if obstacle_in_right:
                w = np.deg2rad(10)
            elif collinear(-2.96, -2.5, robotConfig[0], robotConfig[1], qgoal[0], qgoal[1]):
                following = False
            elif following:
                v = .1
                w = np.deg2rad(-30)
            

        # Cinemática Inversa
        wr = ((2.0 * v) + (w * L)) / (2.0 * r)
        wl = ((2.0 * v) - (w * L)) / (2.0 * r)    
        
        # Enviando velocidades
        sim.simxSetJointTargetVelocity(clientID, l_wheel, wl, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(clientID, r_wheel, wr, sim.simx_opmode_oneshot_wait)
        it += 1
        print(following)
        # print(robotPos)
        # print(robotOri)
       
    sim.simxSetJointTargetVelocity(clientID, r_wheel, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, l_wheel, 0, sim.simx_opmode_oneshot_wait)
        
    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')